# CSS Lab: Extra Topics in CSS

## Section 1: Background

Much of the data studied in other labs is digital trace data from online platforms. However, computational methods can be used to study many types of data. In this lab, you will examine records of taxi trips, compiled by the New York City [Taxi and Limousine Commission](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). Many cities compile data on city functions and make them available through APIs and open data portals.

Taxi trips vary from data studied so far because they combine space, time, and network data. In NYC geographic ___zones___ are used to classify the pick-up and drop-off location of each trip. Taxi trips form a network of interconnections between zones, with each pick-up and drop-off representing a point in space and time.

In this lab, you will combine the taxi data with US Census data to study how socioeconomic factors relate to interconnections between taxi zones.

## Section 2: Imports

In [ ]:
import bz2
import cartopy.io.shapereader as shpreader
from datetime import datetime
from descartes import PolygonPatch
import fiona
import io
import math
import matplotlib as mpl
from matplotlib.collections import PatchCollection
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import scipy.stats as spstats
import shapefile
from shapely.geometry import MultiPoint, MultiPolygon, Polygon, Point, shape
import sys
from tqdm import tqdm

%matplotlib inline

## Section 3: Data
### 3.1 American Community Survey Data
- The American Community Survey (ACS) is a part of the US Census that gives us more detailed information about the US population. We will use some ACS data to get more information about the neighborhoods where taxis go. 

In [ ]:
# Load raw census data
df_census = pd.read_csv('census.tsv', delimiter='\t')

# clean up the data
to_rename = {
    'Families: Median Family Income in the Past 12 Months (In 2016 Inflation-Adjusted Dollars)': 'income',
    'Total Population.1': 'population',
    'Not Hispanic or Latino: White Alone': 'non_hispanic_white',
}

df_census = df_census.rename(columns=to_rename)
df_census['index'] = df_census.FIPS
df_census = df_census.set_index('index')
df_census['non_hispanic_white_frac'] = df_census['non_hispanic_white'] / df_census['population']
df_census = df_census[['FIPS', 'income', 'non_hispanic_white_frac']]
df_census = df_census.dropna()

#reduce memory use by saving data in smallest dtypes
icols = ['FIPS','income']
for c in icols:
    df_census[c] = pd.to_numeric(df_census[c], downcast='integer')
df_census['non_hispanic_white_frac'] = pd.to_numeric(df_census.non_hispanic_white_frac, downcast='float')

df_census.head()

### 3.2 Taxi Trip Data
- Here we load data on all yellow taxi trips in NYC between January 1 and January 17, 2017. 

In [ ]:
df_taxi = pd.read_csv('taxi.csv.gz')
df_taxi.head()

In [ ]:
# clean up data 
df_taxi = df_taxi[df_taxi.tpep_pickup_datetime.notna()]

In [ ]:
#reduce memory use by saving data in the smallest types
icols = ['DOLocationID','PULocationID']
for c in icols:
    df_taxi[c] = pd.to_numeric(df_taxi[c], downcast='integer')
df_taxi['trip_distance'] = pd.to_numeric(df_taxi['trip_distance'], downcast='float')
df_taxi['tpep_pickup_datetime'] = pd.to_datetime(df_taxi.tpep_pickup_datetime)#format='%Y-%m-%d %H:%M:%S'

In [ ]:
# select just the first half of the month's data reduce memory use
df_taxi = df_taxi.sort_values(by='tpep_pickup_datetime')
df_taxi = df_taxi.head(n=int(df_taxi.shape[0]/2))
df_taxi.shape

### 3.3 Load NYC Taxi Zones
- Here we load data on the geographical boundaries of taxi zones, or neighborhoods, in NYC. This will let us draw maps.

In [ ]:
# Helper functions

def log_ticks(low, high):
    lowexp = math.floor(low)
    highexp = math.ceil(high)
    ticks = []
    labels = []
    for e in range(lowexp, highexp + 1):
        for i in range(1, 10):
            v = i * 10**e
            logv = np.log10(v)
            if logv > lowexp and logv < highexp:
                ticks.append(logv)
                labels.append(v)
            if highexp - lowexp > 2:
                break
    return ticks, labels

def plot_geo_values(zones, key=None, label=None, transform=None, cmap='Blues'):
    
    try:
        #backward compatability with old cartopy
        for z in zones:
            z._geometry = z.geometry[0]
    except:
        pass
    
    if transform is None:
        tf = lambda x: x
        itf = lambda x: x
    elif transform == 'log':
        tf = np.log10
        itf = lambda x: np.power(10, x)
    if key is not None:
        values = [r.attributes[key] for r in zones]
        if transform == 'log':
            # Drop zeros for log transform
            values = [v for v in values if v > 0]
        count_max, count_min = tf(max(values)), tf(min(values))
        count_span = count_max - count_min
    patches = []
    cm = plt.get_cmap(cmap)
    for record in zones:
        if key is not None:
            try:
                raw_value = record.attributes[key]
                if transform == 'log' and raw_value == 0:
                    # Skip 0 values for log
                    continue
                value = tf(raw_value)
            except KeyError:
                if transform == 'log':
                    continue
                else:
                    value = 0
            color = cm((value - count_min) / count_span)
        else:
            color = "#efefef"
        patch = PolygonPatch(record._geometry, fc=color, ec='#555555', lw=0.2, alpha=1, zorder=1)
        patches.append(patch)

    mp = MultiPolygon([
        shape(record._geometry)
        for record in zones])
    minx, miny, maxx, maxy = mp.bounds
    w, h = maxx - minx, maxy - miny
    aspect = w / h

    fig = plt.figure(figsize=(8 * aspect / 0.9, 8))
    ax = fig.add_subplot(111)
    ax.set_xlim(minx - 0.2 * w, maxx + 0.2 * w)
    ax.set_ylim(miny - 0.2 * h, maxy + 0.2 * h)
    ax.set_aspect(1)
    ax.add_collection(PatchCollection(patches, match_original=True))
    
    if key is not None:
        # Add colorbar
        plt.subplots_adjust(right=0.9)
        cax = plt.axes([0.95, 0.1, 0.035, 0.8])
        norm = mpl.colors.Normalize(vmin=count_min, vmax=count_max)
        cb1 = mpl.colorbar.ColorbarBase(cax, cmap=cm,
                                        norm=norm,
                                        orientation='vertical')
        if label is not None:
            cb1.set_label(label)
        else:
            cb1.set_label(key)
        if transform == 'log':
            ticks, labels = log_ticks(count_min, count_max)
            cb1.set_ticks(ticks)
            cb1.set_ticklabels(labels)

In [ ]:
reader = shpreader.Reader('taxi/taxi_zones/taxi_zones-latlong.shp')
taxi_geo = list(reader.records())
plot_geo_values(taxi_geo)

### 3.4 Load Census Tracts
- here we load data from the US census on geographical areas, called "census tracts." Note that census tracts are different than taxi zones. In general, every data source that does geographic things or mapping uses its own way of dividing up space, which makes doing work with maps and geography especially hard.

In [ ]:
reader = shpreader.Reader('tracts/cb_2017_36_tract_500k.shp')
tract_geo = list(reader.records())

# Keep tracts in NYC
geoids = set(df_census['FIPS'])
tract_geo = [
    record for record in tract_geo
    if int(record.attributes['GEOID']) in geoids]

plot_geo_values(tract_geo)

## Section 4. Visualize Data
#### Count Pick-Ups and Drop-Offs

Let's start by visualizing the data. First, we use a helper function to count the number of pick-ups and drop-offs in each taxi zone.

In [ ]:
# Helper functions

def annotate_records(records, data, key):
    for r in records:
        geoid = int(r.attributes['GEOID'])
        r.attributes[key] = data[geoid]

def update_counts(df, records):
    print('Updating counts')
    counts_pickup = df.groupby('PULocationID').count()['tpep_pickup_datetime']
    counts_dropoff = df.groupby('DOLocationID').count()['tpep_pickup_datetime']
    zone_pu_counts = {}
    zone_do_counts = {}
    for zone, count in dict(counts_pickup).items():
        zone_pu_counts[zone] = count
    for zone, count in dict(counts_dropoff).items():
        zone_do_counts[zone] = count
    for r in records:
        geoid = r.attributes['LocationID']
        r.attributes['pickups'] = zone_pu_counts.get(geoid, 0)    
        r.attributes['dropoffs'] = zone_do_counts.get(geoid, 0)    


In [ ]:
update_counts(df_taxi, taxi_geo)
taxi_geo[0].attributes

#### Visualize Pick-Ups
The following cell plots a map of New York taxi zones, with darker zones representing more taxi pick-ups. The number of pick-ups exhibits a heavy tail, so the colors are chosen based on the logarithm of the counts.

In [ ]:
plot_geo_values(taxi_geo, 'pickups', 'Pick-Ups', transform='log')

#### Remove Airports
In the above visualization, the taxi zones representing airports stand out as outliers. We can remove them from the analysis and update trip counts using the following code.

In [ ]:
# Identify airport zones
airports = [r for r in taxi_geo if 'Airport' in r.attributes['zone']]
airport_zones = [r.attributes['LocationID'] for r in airports]

# Remove airports zones and trips
taxi_geo = [r for r in taxi_geo if 'Airport' not in r.attributes['zone']]

In [ ]:
df_taxi = df_taxi[
    (~df_taxi['DOLocationID'].isin(airport_zones))
    & (~df_taxi['PULocationID'].isin(airport_zones))]

# Update counts and plot
update_counts(df_taxi, taxi_geo)

In [ ]:
plot_geo_values(taxi_geo, 'pickups', transform='log')

#### Short Answer 1
Whether or not to include airports depends on the type of questions we want to answer. Most people travelling to and from airports do not live or work at that airport. For each of the following two examples, explain why you would or would not include trips to and from airports in an analysis:

1. Understanding wear and tear on roads.
2. Understanding relationships between residential communities.

🤔 Your answer here:

#### Visualize income
Now let's visualize household income. This data is organized into census tracts, which are typically smaller than taxi zones.

In [ ]:
# Helper functions

# Get geoid->values from df for each element in geo
def get_geo_map(df, key, geo):
    result = {}
    for i, r in enumerate(geo):
        geoid = int(r.attributes['GEOID'])
        result[geoid] = df.loc[geoid, key]
    return result

In [ ]:
tract_income_map = get_geo_map(df_census, 'income', tract_geo)
annotate_records(tract_geo, tract_income_map, 'income')
plot_geo_values(tract_geo, 'income', transform='log')

#### Short Answer 2
Do the areas with high income tend to have a higher than average, near average, or below average number of taxi pick-ups? (Hint: compare the income map you just made with the trip maps you made earlier.)

What is one way income level could influence the number of taxi trips to or from a neighborhood?

🤔 Your answer here:

## Pairwise Trip Data
Rather than individual zone pick-ups, we're really interested in connections _between_ zones. In other words, we want to analyze the pairwise trip data.

#### Create data frame for pairwise data
The first step is to create a new data frame. The entries of this data frame are indexed by pairs of zones rather than single zones.

In [ ]:
# Helper functions

def make_pair_df(geo):
    index = set([
        (r.attributes['LocationID'], s.attributes['LocationID'])
        for r in geo
        for s in geo
        if r.attributes['LocationID'] != s.attributes['LocationID']
    ])
    df_pairs = pd.DataFrame(index=list(index))
    return df_pairs

def plot_pairs(
    geo,
    df_pairs,
    key,
    label=None,
    bounds=None,
    threshold=0,
    transform=None,
    linewidth=0.1,
    alpha=0.3
):
    points = {}
    patches = []
    print('Finding representative points')
    sys.stdout.flush()
    for record in tqdm(geo):
        patch = PolygonPatch(record._geometry, fc='#efefef', ec='#555555', lw=0.2, alpha=1, zorder=1)
        patches.append(patch)
        record_id = record.attributes['LocationID']
        points[record_id] = record._geometry.representative_point().coords[0]

    mp = MultiPolygon([
        shape(record._geometry)
        for record in geo])
    minx, miny, maxx, maxy = mp.bounds
    w, h = maxx - minx, maxy - miny

    if bounds is None:
        xlim = (minx - 0.2 * w, maxx + 0.2 * w)
        ylim = (miny - 0.2 * h, maxy + 0.2 * h)
    else:
        xlim = (bounds[0], bounds[1])
        ylim = (bounds[2], bounds[3])
    ar = (xlim[1] - xlim[0]) / (ylim[1] - ylim[0])
    
    fig = plt.figure(figsize=(8 * ar, 8))
    ax = fig.add_subplot(111)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.add_collection(PatchCollection(patches, match_original=True))
    
    if transform is None:
        tf = lambda x: x
        itf = lambda x: x
    if transform == 'log':
        tf = lambda x: np.where(x>0, np.log10(x), 0)
        itf = lambda x: np.power(10, x)
        
               
    trip_max = tf(df_pairs['trips'].max())
    
    cm = plt.get_cmap('afmhot')
    sys.stdout.flush()
    print('Drawing lines')
    sys.stdout.flush()
    for a_id, b_id in tqdm(df_pairs.index):
        # Combine both directions for plot
        if a_id >= b_id:
            continue
        a_point = points[a_id]
        b_point = points[b_id]
        try:
            raw_count = (
                df_pairs['trips'][(a_id, b_id)]
                + df_pairs['trips'][(b_id, a_id)])
            try:
                if raw_count < threshold:
                    continue
            except ValueError:
                print("raw", raw_count, 't', threshold, 'a', a_id, 'b', b_id)
                raise
            with np.errstate(divide='ignore'):
                count = tf(raw_count)
            f = count / trip_max
            color = "#000000"
            l = mlines.Line2D(
                [a_point[0], b_point[0]],
                [a_point[1], b_point[1]],
                color='#000000',
                linewidth=linewidth,
                alpha=f*alpha)
            ax.add_line(l)
        except RuntimeError:
            pass

In [ ]:
df_pairs = make_pair_df(taxi_geo)
df_pairs.head()

#### Count number of trips
Now we can count the number of trips between each pair of zones.

In [ ]:
# Helper function

# Calculate number of trips
def count_trips(df_taxi, df_pairs):
    print('Counting trips')
    index = df_pairs.index
    df_count = df_taxi.groupby(['PULocationID', 'DOLocationID']).count()['tpep_pickup_datetime']
    df_pairs['trips'] = np.zeros(len(index))
    for a_id, b_id in tqdm(df_pairs.index):
        # Sum trips in both directions
        count = df_count.get((a_id, b_id), 0)
        df_pairs.trips[(a_id, b_id)] = count

In [ ]:
count_trips(df_taxi, df_pairs)
#plt.hist(df_pairs.trips, bins=30)
#plt.xlabel('Trips Between Zones')
#plt.ylabel('Number of Zone Pairs')
df_pairs.sort_values('trips', ascending=False).head()

#### Visualize Trips
The following visualization draws a line between each pair of zones, with darker lines representing more traffic.

In [ ]:
plot_pairs(
    taxi_geo, df_pairs, 'trips',
    label='Trips',
    bounds=(-74.07, -73.83, 40.62, 40.93),
    transform='log')

#### Adding a Threshold
The above visualization is a little difficult to interpret because there are so many lines. One way to reduce clutter is to add a threshold, only drawing lines between zones with a minimum number of trips.

In [ ]:
threshold = 10000
plot_pairs(
    taxi_geo, df_pairs, 'trips',
    label='Trips',
    bounds=(-74.07, -73.83, 40.62, 40.93),
    threshold=threshold,
    linewidth=1,
    alpha=0.5,
    transform='log')

#### Short Answer 3

What do the networks of taxi trips tell us about the relationships among neighborhoods? Explain your answer in terms of boroughs, trip length, and network hubs. 

🤔 Your answer here:

#### Calculate Average Trip Distance

In [ ]:
def get_mean_distance(df_taxi, df_pairs):
    print('Finding mean distance')
    # Calculate mean trip distance
    df_mean = df_taxi.groupby(['PULocationID', 'DOLocationID']).mean()
    df_trip = df_mean['trip_distance']
    df_pairs['trip_distance'] = np.zeros(len(df_pairs.index))
    for a_id, b_id in tqdm(df_pairs.index):
        # Get counts in each direction
        ab_count = df_pairs['trips'].get((a_id, b_id), 0)
        ba_count = df_pairs['trips'].get((b_id, a_id), 0)
        # If no trips, average distance is undefined
        if (ab_count + ba_count == 0):
            df_pairs.at[(a_id, b_id), 'trip_distance'] = float('nan')
        # Otherwise take a weighted average of each direction
        else:
            df_pairs.at[(a_id, b_id), 'trip_distance'] = (
               ab_count * df_trip.get((a_id, b_id), 0)
                + ba_count * df_trip.get((b_id, a_id), 0)) / (ab_count + ba_count)

In [ ]:
get_mean_distance(df_taxi, df_pairs)

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1, 2, 1);
df_pairs['trip_distance'].hist(bins=30)
plt.xlabel("Miles")
plt.ylabel("Number of Zone Pairs")
plt.subplot(1, 2, 2);
plt.hist(df_taxi[df_taxi['trip_distance'] < 40]['trip_distance'], bins=30)
plt.xlabel("Miles")
plt.ylabel("Number of Trips")
plt.grid()
plt.tight_layout()

#### Compute a map from census tract values to taxi zone
- here is where we match up the census tracts with the taxi zones. You don't need to worry how we do this for the lab, but this code may be a useful reference if you want to do similar things in the future.

In [ ]:
# Helper functions

# Create a function mapping census tract vectors to taxi zone vectors
def get_tract_to_zone(taxi_geo, tract_geo):
    # Get fractions of zone overlapping each tract
    weight_zone_tract, all_zones = get_weight_zone_tract(taxi_geo, tract_geo)
    # Create a function to map census data to taxi zones
    def tract_to_zone(tract_data):
        zone_data = pd.Series(index=list(all_zones))
        for zone_id in tqdm(all_zones):
            zone_data[zone_id] = 0
            for tract_id, d in tract_data.items():
                f = weight_zone_tract[zone_id][tract_id]
                zone_data[zone_id] += f * tract_data[tract_id]
        return zone_data
    # Return function
    return tract_to_zone

# Create a function mapping census tract vectors to taxi zone vectors
def get_weight_zone_tract(taxi_geo, tract_geo):
    
    # Calculate the fraction of each zone that overlaps with a given tract
    weight_zone_tract = {}
    empty_zones = set()
    all_zones = set()
    for zone in tqdm(taxi_geo):
        zone_id = zone.attributes['LocationID']
        all_zones.add(zone_id)
        weight_tract = {}
        for tract in tract_geo:
            tract_id = int(tract.attributes['GEOID'])
            # This is a rough approximation that treats lat/long as cartesian
            f = zone._geometry.intersection(tract._geometry).area / zone._geometry.area
            weight_tract[tract_id] = f
        # Exclude taxi zones without enough data
        zone_total = sum(weight_tract.values())
        if zone_total < 0.5:
            empty_zones.add(zone_id)
        else:
            # Normalize
            for tract_id in weight_tract.keys():
                weight_tract[tract_id] /= zone_total
        weight_zone_tract[zone_id] = weight_tract
    return weight_zone_tract, all_zones


In [ ]:
tract_to_zone = get_tract_to_zone(taxi_geo, tract_geo)

#### Taxi Zone Income
- here we show average income for people living in each taxi zone.

In [ ]:
zone_incomes = tract_to_zone(df_census['income'])

In [ ]:
for zone in tqdm(taxi_geo): 
    zone_id = zone.attributes['LocationID']
    zone.attributes['income'] = zone_incomes[zone_id]
plot_geo_values(taxi_geo, 'income', label="Income")

The following histogram shows the distribution of incomes for each taxi zone in the city. Note that some neighborhoods have very high average incomes, while others have very low incomes.

In [ ]:
plt.hist(zone_incomes, bins=24)
plt.xlabel('Income')
plt.ylabel('Number of Zones')

#### Income Difference
- Here we compare the incomes of neighborhoods where taxi trips start with the incomes of the neighborhoods where they end. 

In [ ]:
# Helper functions

# Calculate trips vs income difference
def get_log_diff(df_pairs, attribute, values, min_value=0):
    print('Finding attribute difference')
    key = 'log_{}_diff'.format(attribute)
    df_pairs[key] = np.zeros(len(df_pairs.index))
    for i, j in tqdm(df_pairs.index):
        a = values[i]
        b = values[j]
        # Ignore zero case and partial zones
        if a > min_value and b > min_value:
            df_pairs.at[(i, j), key] = np.log10(b) - np.log10(a)
        else:
            df_pairs.at[(i, j), key] = float('nan')

def get_diff(df_pairs, attribute, values, min_value=0):
    print('Finding attribute difference')
    key = '{}_diff'.format(attribute)
    df_pairs[key] = np.zeros(len(df_pairs.index))
    for i, j in tqdm(df_pairs.index):
        a = values[i]
        b = values[j]
        if a > min_value and b > min_value:
            df_pairs.at[(i, j), key] = b - a
        else:
            df_pairs.at[(i, j), key] = float('nan')


In [ ]:
min(df_census['income'])

In [ ]:
df_pairs.head()

In [ ]:
get_diff(df_pairs, 'income', zone_incomes, min_value=12300)
df_pairs['income_diff'].abs().hist(bins=30)
plt.xlabel('Difference in income (dollars)')
plt.ylabel('Number of pairs')

#### Comparing income and connectivity

In [ ]:
# Helper functions

def plot_binned(x, y, alpha=1, bins=30, log=None, xlabel=None, ylabel=None, title=None):
    # Remove nan
    isnan = np.isnan(x) | np.isnan(y)
    x = np.array(x)[~isnan]
    y = np.array(y)[~isnan]
    num_bins = bins
    low = min(x)
    high = max(x)
    delta = (high - low) / num_bins
    bin_x = [[] for i in range(num_bins)]
    bin_y = [[] for i in range(num_bins)]
    bin_cx = [low + delta * (i + 1/2) for i in range(num_bins)]
    for i in range(len(x)):
        try:
            b = int(math.floor((x[i] - low) / delta))
        except ValueError:
            print(x[i] == 'nan', x[i], low, delta)
            raise
        if b == num_bins:
            # Include edge case on top bin
            b -= 1
        bin_x[b].append(x[i])
        bin_y[b].append(y[i])
    means = np.array([np.mean(by) for by in bin_y])
    std = np.array([np.std(by) for by in bin_y])
    count = np.sqrt(np.array([len(by) for by in bin_y]))
    se = std / count
    low95 = means - 2 * se
    high95 = means + 2 * se
    if log == 'y' or log == 'xy':
        low95[low95 <= 0] = 0.1
        high95[high95 <= 0] = 0.1
    plt.fill_between(bin_cx, low95, high95, color='#dfdfdf', zorder=2, alpha=alpha)
    if log == 'x':
        plt.semilogx(x, y, '.', zorder=1, alpha=0.1)
        plt.semilogx(bin_cx, means)
        plt.semilogx(bin_cx, means, '--', color='#999999')
        plt.semilogx(bin_cx, means + 2 * se, '--', color='#999999')
    elif log == 'y':
        plt.semilogy(x, y, '.', zorder=1, alpha=0.1)
        plt.semilogy(bin_cx, means)
        plt.semilogy(bin_cx, low95, '--', color='#999999')
        plt.semilogy(bin_cx, high95, '--', color='#999999')
    elif log == 'xy':
        plt.loglog(x, y, '.', zorder=1, alpha=0.1)
        plt.loglog(bin_cx, means)
        plt.loglog(bin_cx, low95, '--', color='#999999')
        plt.loglog(bin_cx, high95, '--', color='#999999')
    else:
        plt.plot(x, y, '.', zorder=1, alpha=0.1)
        plt.plot(bin_cx, means)
        plt.plot(bin_cx, means - 2 * se, '--', color='#999999')
        plt.plot(bin_cx, means + 2 * se, '--', color='#999999')
    if xlabel is not None:
        plt.xlabel(xlabel)
    if ylabel is not None:
        plt.ylabel(ylabel)
    if title is not None:
        plt.title(title)

def filter_borough(df_pairs, borough):
    borough_ids = set()
    for r in taxi_geo:
        if r.attributes['borough'] == 'Manhattan':
             borough_ids.add(int(r.attributes['LocationID']))
    in_borough = list()
    for s, t in df_pairs.index:
        if s in borough_ids and t in borough_ids:
            in_borough.append(True)
        else:
            in_borough.append(False)
    df_pairs_borough = df_pairs[in_borough]
    return df_pairs_borough

In [ ]:
df_pairs_nyc = filter_borough(df_pairs, 'Manhattan')
print("Pairs in full set:", len(df_pairs))
print("Pairs in Manhattan:", len(df_pairs_nyc))

#### See the number of trips between neighborhoods according to the difference in income between the neighborhoods

In [ ]:
plt.figure(figsize=(8, 8))
plt.ylim([0,10000])
plot_binned(
    df_pairs_nyc['income_diff'], df_pairs_nyc['trips'],
    #log='y', 
    alpha=0.6,
    xlabel='Difference in income',
    ylabel='Number of Trips')


####  See the number of trips between neighborhoods according to the sinstance between the neighborhoods

In [ ]:
# Calculate trip distance vs income difference
plt.figure(figsize=(8, 8))
plot_binned(
    df_pairs_nyc['trip_distance'], df_pairs_nyc['trips'],
    #log='xy', 
    alpha=0.8, bins=10,
    xlabel='Mean ride distance between zones (miles)',
    ylabel='Number of trips between zones')
plt.ylim([0, 10000])

### But wait, maybe distance and income are related.
- Neighborhoods that are close together may have more similar income than neighborhoods that are far apart. 
- Maybe the relationship we saw between trips and income was just a side effect of the distance between neighborhoods, and not really a result of their incomes. 
- We can test this by only looking at trips of a specific length, say 1 mile or 7 miles, and seeing whether income influences where people go even when they're going the same distance.

In [ ]:
# Helper functions
def filter_zones(df_pairs, distance, tolerance):
    df = df_pairs[(df_pairs.trip_distance - distance).abs() < tolerance]
    return df

In [ ]:
plt.figure(figsize=(12, 4))
for i, miles in enumerate([1, 4, 7]):
    df_pairs_miles = filter_zones(df_pairs_nyc, miles, 1.5)
    plt.subplot(1, 3, i + 1)
    plot_binned(
        df_pairs_miles['income_diff'],
        df_pairs_miles['trips'],
        bins=10, #log='y',
        alpha=0.6,
        xlabel='Difference in income',
        ylabel='Number of Trips',
        title='{} Miles'.format(miles))
    #plt.ylim([10, 10000])
    #plt.xlim([-1, 1])
plt.tight_layout()

#### Time of Day
- Another factor affects taxi trips: time of day. 7am taxi trips are probably different than 10pm taxi trips, for example. 

### Short answer 4:
- Give three examples of how different times of day might affect who is riding taxis and where they are going. Explain how each example might look in the maps of taxi traffic we made earlier. 

Your answer here:

In [ ]:
# Helper functions

def get_trip_datetime(df):
    print('Finding trip times')
    
    date = pd.to_datetime(df.tpep_pickup_datetime, ) #format='%Y-%m-%d %H:%M:%S'
    
    df['weekday'] = date.dt.weekday
    df['hour'] = date.dt.hour
    df['minute'] = date.dt.minute
    return

def get_time_pairs(df, taxi_geo, zone_incomes):
    time_periods = {
        'AM Rush': [(5, 10)],
        'Midday': [(10, 16)],
        'PM Rush': [(16, 20)],
        'Night': [(0, 5), (20, 24)]
    }
    pairs = {}
    for label, periods in time_periods.items():
        selection = pd.Series(False, index=df.index)
        # Check hour of day
        for start, stop in periods:
            selection = selection | (
                (df.hour >= start)
                & (df.hour < stop))
            print(len(df[selection]))
        # M-F only
        selection = selection & (df.weekday < 5)
        # Select data
        df_period = df[selection]
        pairs[label] = make_pair_df(taxi_geo)
        count_trips(df_period, pairs[label])
        get_mean_distance(df_period, pairs[label])
        get_log_diff(pairs[label], 'income', zone_incomes, min_value=12300)
    return pairs

In [ ]:
# Helper functions
def get_time_pairs(df, taxi_geo, zone_incomes):
    time_periods = {
        'AM Rush': [(5, 10)],
        'Midday': [(10, 16)],
        'PM Rush': [(16, 20)],
        'Night': [(0, 5), (20, 24)]
    }
    pairs = {}
    for label, periods in time_periods.items():
        selection = pd.Series(False, index=df.index)
        # Check hour of day
        for start, stop in periods:
            selection = selection | (
                (df.tpep_pickup_datetime.dt.hour >= start)
                & (df.tpep_pickup_datetime.dt.hour < stop))
            print(len(df[selection]))
        # M-F only
        selection = selection & (df.tpep_pickup_datetime.dt.weekday < 5)
        # Select data
        df_period = df[selection]
        pairs[label] = make_pair_df(taxi_geo)
        count_trips(df_period, pairs[label])
        get_mean_distance(df_period, pairs[label])
        get_diff(pairs[label], 'income', zone_incomes, min_value=12300)
    return pairs

In [ ]:
df = df_taxi[~df_taxi.index.duplicated(keep='first')]
#get_trip_datetime(df)
df.columns

In [ ]:
df.head()

In [ ]:
time_pairs = get_time_pairs(df, taxi_geo, zone_incomes)

In [ ]:
plt.figure(figsize=(8, 8))
for i, (label, pairs) in enumerate(time_pairs.items()):
    plt.subplot(2,2,1 + i)
    plot_binned(
        pairs['income_diff'], pairs['trips'],
        #log='y', 
        alpha=0.8,
        xlabel='Difference in income',
        ylabel='Number of Trips',
        title=label)
    #plt.ylim([0.5, 100])
plt.tight_layout()

In [ ]:
df_weekend = df[df.tpep_pickup_datetime.dt.weekday >= 5]
df_pairs_weekend = make_pair_df(taxi_geo)
count_trips(df_weekend, df_pairs_weekend)
get_mean_distance(df_weekend, df_pairs_weekend)
get_diff(df_pairs_weekend, 'income', zone_incomes, min_value=12300)


In [ ]:
plt.figure(figsize=(12, 8))

for i, miles in enumerate([1, 4, 7]):
    df_pairs_miles = filter_zones(df_pairs_weekend, miles, 1.5)
    plt.subplot(2, 3, i + 1)
    plot_binned(
        df_pairs_miles['income_diff'],
        df_pairs_miles['trips'],
        bins=15, #log='y', 
        alpha=0.8,
        xlabel='Difference in income',
        ylabel='Number of Trips',
        title='{} Miles (weekend)'.format(miles))
    #plt.ylim([0.9, 10000])
    #plt.xlim([-1, 1])
    
for i, miles in enumerate([1, 4, 7]):
    df_pairs_miles = filter_zones(time_pairs['AM Rush'], miles, 1.5)
    plt.subplot(2, 3, i + 4)
    plot_binned(
        df_pairs_miles['income_diff'],
        df_pairs_miles['trips'],
        bins=15, #log='y', 
        alpha=0.8,
        xlabel='Difference in income',
        ylabel='Number of Trips',
        title='{} Miles (rush hour)'.format(miles))
    #plt.ylim([0.9, 10000])
    #plt.xlim([-1, 1])
    
plt.tight_layout()


#### Examining individual pairs
- To understand these patterns more, let's look at some specific neighborhoods and the trips between them.

In [ ]:
# Helper functions

def print_zone(geo, location_id):
    for r in geo:
        if r.attributes['LocationID'] == location_id:
            print('{}: {}'.format(
                r.attributes['borough'],
                r.attributes['zone']))
            print('Median income:', r.attributes['income'])
            break
            
def trip_weekday_hist(df, geo, source, dest, ymax):
    r_source = taxi_geo_dict[source]
    r_dest = taxi_geo_dict[dest]
    bins = [-0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5]
    df[(df.PULocationID == source) & 
       (df.DOLocationID == dest)
      ].tpep_pickup_datetime.dt.weekday.hist(bins=bins)
    plt.ylim([0, ymax])
    plt.title('{} \n-> {}'.format(
        r_source.attributes['zone'],
        r_dest.attributes['zone']))
    plt.xticks(
        range(7),
        ['M', 'Tu', 'W', 'Th', 'F', 'Sa', 'Su'])
    plt.xlim([-0.5,6.5])

def trip_time_hist(df, geo, source, dest, ymax):
    r_source = taxi_geo_dict[source]
    r_dest = taxi_geo_dict[dest]
    df[(df.PULocationID == source) & 
       (df.DOLocationID == dest)
      ].tpep_pickup_datetime.dt.hour.hist(bins=24)
    plt.ylim([0, ymax])
    plt.title('{} \n-> {}'.format(
        r_source.attributes['zone'],
        r_dest.attributes['zone']))
    
taxi_geo_dict = {}
for r in taxi_geo:
    taxi_geo_dict[r.attributes['LocationID']] = r

#### neighborhoods with similar income
- This code gets a list of neighborhoods with **simiar** incomes that have the most trips between them.
- The neighborhood ID numbers appear on the left on the list in parentheses, e.g. `(262, 162)` means all trips between neighborhood with ID 262 and neighborhood with ID 162.

In [ ]:
sel = (((time_pairs['AM Rush'].income_diff).abs() < 41339)
       & (time_pairs['AM Rush'].trip_distance > 2))

time_pairs['AM Rush'][sel].sort_values('trips', ascending=False).head().round(2)

- This code lets you look up a neighborhood's name and income based on its ID number. 
- Try putting in some of the neighborhood ID numbers from above.

In [ ]:
print_zone(taxi_geo, 148)

In [ ]:
print_zone(taxi_geo, 162)

- This code gets a list of neighborhoods with very **different** incomes that have the most trips between them.

In [ ]:
sel = (((time_pairs['AM Rush'].income_diff).abs() > 41339)
       & (time_pairs['AM Rush'].trip_distance > 2))

time_pairs['AM Rush'][sel].sort_values('trips', ascending=False).head()

In [ ]:
print_zone(taxi_geo, 229)

In [ ]:
print_zone(taxi_geo, 75)

### Examine trips between two specific neighborhoods by time of day and day of week
- Change the neighborhood ID numbers in the cell below to a pair of neighborhoods that you are interested in. 
- Then run the code below them to see which direction people travel between them, and when.

In [ ]:
neighborhood_a = 75
neighborhood_b = 229

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
trip_weekday_hist(df, taxi_geo, source=neighborhood_a, dest=neighborhood_b, ymax=300)  
plt.ylabel('trips')
plt.subplot(1,2,2)
trip_weekday_hist(df, taxi_geo, source=neighborhood_b, dest=neighborhood_a, ymax=300)    
plt.tight_layout()

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
trip_time_hist(df, taxi_geo, source=neighborhood_a, dest=neighborhood_b, ymax=130)
plt.ylabel('Trips')
plt.xlabel('Hour of the day')
plt.subplot(1,2,2)
trip_time_hist(df, taxi_geo, source=neighborhood_b, dest=neighborhood_a, ymax=130)
plt.xlabel('Hour of the day')


## Try It Yourself: race and ethnicity
- Income is not the only information we have about neighborhoods. 
- In the next section, you'll do the same analysis as before, but this time with data about the proportion of a neighborhood that identifies as white and not Hispanic. That is, instead of asking mow many taxi trips connect neighborhoods with different income levels, we will ash how many taxi trips connect neighborhoods with different proprotions of white people and people of color. 

### Short answer 5:
- What do you expect the relationship between neighborhoods' racial composition and taxi trips to look like? Explain why.

Your answer here:

#### First, a map of NYC showing the data

In [ ]:
zone_race = tract_to_zone(df_census['non_hispanic_white_frac'])
for zone in taxi_geo: 
    zone_id = zone.attributes['LocationID']
    zone.attributes['non_hispanic_white_frac'] = zone_race[zone_id]
plot_geo_values(taxi_geo, 'non_hispanic_white_frac', 
                label="Fraction Non-Hispanic White", 
                cmap='spring')

In [ ]:
get_diff(df_pairs, 'non_hispanic_white_frac', zone_race)

#### trips according to race/ethnicity

In [ ]:
plt.figure(figsize=(8, 8))
plot_binned(
    df_pairs['non_hispanic_white_frac_diff'], df_pairs['trips'],
    #log='y',
    alpha=0.8,
    xlabel='Difference in Fraction Non-Hispanic White',
    ylabel='Number of Trips')
plt.ylim([0,5000])

#### Short answer 6:
- Was your prediction correct? Explain why or why not. 

Your answer here:

# Reflection Questions
1. We showed a number of factors that all influence taxi trips, including income, race, distance, time of day, and day of week. Name one other factor that might influence taxi trips. Explain how you would measure that using this data. Then explain how it is (or is not) related to the factors we looked at in the lab.

2. Pick a city or town that you know that is not NYC. How might the taxi trip data look different there? In what ways might it look similar? Explain why.

3. Taxis are one way to get around a city, but New Yorkers get around many ways including subways, ferries, walking, biking, driving, busses, and even (for the very wealthy) helecopters. What is one research question that taxi data would not be very helpful for answering? Why? What is one research question that taxi data would be useful for answering? Why? 

Your answers here